In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import numpy as np
import pandas as pd
from time import time, sleep
from datetime import datetime

In [2]:
# Create directories for saving images
import os
CAM_1_left_images_dir = 'CAM_1_left_images'
CAM_1_right_images_dir = 'CAM_1_right_images'
CAM_1_depth_images_dir = 'CAM_1_depth_images'
os.makedirs(CAM_1_left_images_dir, exist_ok=True)
os.makedirs(CAM_1_right_images_dir, exist_ok=True)
os.makedirs(CAM_1_depth_images_dir, exist_ok=True)

CAM_2_left_images_dir = 'CAM_2_left_images'
CAM_2_right_images_dir = 'CAM_2_right_images'
CAM_2_depth_images_dir = 'CAM_2_depth_images'
os.makedirs(CAM_2_left_images_dir, exist_ok=True)
os.makedirs(CAM_2_right_images_dir, exist_ok=True)
os.makedirs(CAM_2_depth_images_dir, exist_ok=True)

In [3]:
class SimpleDataCollector(Node):
    def __init__(self):
        super().__init__('simple_data_collector')
        ##############################################################################
        self.bridge = CvBridge()
        self.last_timestamp = 0
        self.sync_rate = 1/100
        ##############################################################################
        self.CAM_1_left_image = None
        self.CAM_1_right_image = None
        self.CAM_1_depth_image = None
        self.CAM_2_left_image = None
        self.CAM_2_right_image = None
        self.CAM_2_depth_image = None
        ##############################################################################
        self.CAM_1_left_time = 0
        self.CAM_1_right_time = 0
        self.CAM_1_depth_time = 0
        self.CAM_2_left_time = 0
        self.CAM_2_right_time = 0
        self.CAM_2_depth_time = 0
        ##############################################################################
        self.CAM_1_left_image_sub = self.create_subscription(Image, '/cam_1/zed_node/left/image_rect_color', self.CAM_1_left_image_callback, 10)
        self.CAM_1_right_image_sub = self.create_subscription(Image, '/cam_1/zed_node/right/image_rect_color', self.CAM_1_right_image_callback, 10)
        self.CAM_1_depth_image_sub = self.create_subscription(Image, '/cam_1/zed_node/depth/depth_registered', self.CAM_1_depth_image_callback, 10)
        self.CAM_2_left_image_sub = self.create_subscription(Image, '/cam_2/zed_node/left/image_rect_color', self.CAM_2_left_image_callback, 10)
        self.CAM_2_right_image_sub = self.create_subscription(Image, '/cam_2/zed_node/right/image_rect_color', self.CAM_2_right_image_callback, 10)
        self.CAM_2_depth_image_sub = self.create_subscription(Image, '/cam_2/zed_node/depth/depth_registered', self.CAM_2_depth_image_callback, 10)
        
    def synchronization(self):
        #if self.left_image is not None and self.right_image is not None and self.depth_image is not None:

        if self.CAM_1_left_time == self.CAM_1_right_time == self.CAM_1_depth_time == self.CAM_2_left_time == self.CAM_2_right_time == self.CAM_2_depth_time:
            ##########################################################################
            cv_CAM_1_left = self.bridge.imgmsg_to_cv2(self.CAM_1_left_image, desired_encoding='bgr8')
            cv_CAM_1_right = self.bridge.imgmsg_to_cv2(self.CAM_1_right_image, desired_encoding='bgr8')
            cv_CAM_1_depth = self.bridge.imgmsg_to_cv2(self.CAM_1_depth_image, desired_encoding='passthrough')
            cv_CAM_2_left = self.bridge.imgmsg_to_cv2(self.CAM_2_left_image, desired_encoding='bgr8')
            cv_CAM_2_right = self.bridge.imgmsg_to_cv2(self.CAM_2_right_image, desired_encoding='bgr8')
            cv_CAM_2_depth = self.bridge.imgmsg_to_cv2(self.CAM_2_depth_image, desired_encoding='passthrough')
            ##########################################################################
            CAM_1_left_filename = f"{CAM_1_left_images_dir}/frame_{self.CAM_1_left_time}.png"
            CAM_1_right_filename = f"{CAM_1_right_images_dir}/frame_{self.CAM_1_right_time}.png"
            CAM_1_depth_filename = f"{CAM_1_depth_images_dir}/frame_{self.CAM_1_depth_time}.npy"
            CAM_2_left_filename = f"{CAM_2_left_images_dir}/frame_{self.CAM_2_left_time}.png"
            CAM_2_right_filename = f"{CAM_2_right_images_dir}/frame_{self.CAM_2_right_time}.png"
            CAM_2_depth_filename = f"{CAM_2_depth_images_dir}/frame_{self.CAM_2_depth_time}.npy"
            ##########################################################################
            cv2.imwrite(CAM_1_left_filename, cv_CAM_1_left)
            cv2.imwrite(CAM_1_right_filename, cv_CAM_1_right)
            np.save(CAM_1_depth_filename, cv_CAM_1_depth)
            cv2.imwrite(CAM_2_left_filename, cv_CAM_2_left)
            cv2.imwrite(CAM_2_right_filename, cv_CAM_2_right)
            np.save(CAM_2_depth_filename, cv_CAM_2_depth)
            ###########################################################################
            print('Current time is:' + str(self.CAM_1_left_time))
            interval = self.CAM_1_left_time - self.last_timestamp
            print('Time   interval:' + str(interval))
            ###########################################################################
            self.last_timestamp = self.CAM_1_left_time


    ###################################################################################
    def CAM_1_left_image_callback(self, msg):
        self.CAM_1_left_image = msg
        self.CAM_1_left_time = self.CAM_1_left_image.header.stamp.sec + self.CAM_1_left_image.header.stamp.nanosec / 1e9
        self.synchronization()
    def CAM_1_right_image_callback(self, msg):
        self.CAM_1_right_image = msg
        self.CAM_1_right_time = self.CAM_1_right_image.header.stamp.sec + self.CAM_1_right_image.header.stamp.nanosec / 1e9
        self.synchronization()
    def CAM_1_depth_image_callback(self, msg):
        self.CAM_1_depth_image = msg
        self.CAM_1_depth_time = self.CAM_1_depth_image.header.stamp.sec + self.CAM_1_depth_image.header.stamp.nanosec / 1e9
        self.synchronization()
    ###################################################################################
    def CAM_2_left_image_callback(self, msg):
        self.CAM_2_left_image = msg
        self.CAM_2_left_time = self.CAM_2_left_image.header.stamp.sec + self.CAM_2_left_image.header.stamp.nanosec / 1e9
        self.synchronization()
    def CAM_2_right_image_callback(self, msg):
        self.CAM_2_right_image = msg
        self.CAM_2_right_time = self.CAM_2_right_image.header.stamp.sec + self.CAM_2_right_image.header.stamp.nanosec / 1e9
        self.synchronization()
    def CAM_2_depth_image_callback(self, msg):
        self.CAM_2_depth_image = msg
        self.CAM_2_depth_time = self.CAM_2_depth_image.header.stamp.sec + self.CAM_2_depth_image.header.stamp.nanosec / 1e9
        self.synchronization()

In [4]:
rclpy.init()
simple_data_collector = SimpleDataCollector()
rclpy.spin(simple_data_collector)

KeyboardInterrupt: 